<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393/578 Máquinas de Aprendizaje - 2019-1 </h1>

<H3 align='center'> Tarea 2  </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Manipulaciones básicas en numpy
* Preprocesamiento y exploración de datos, creación de datos sinteticos
* Regresión Lineal Regularizada: _Ridge_ y _Lasso_
* SVM
* Nociones de clases desbalanceadas
* Más _Cross Validation_


**Formalidades**  
* Equipos de trabajo de 2 personas (*Ambos estudiantes deben estar preparados para presentar la tarea el día de la entrega*)
* El entregable debe ser un _Jupyter Notebook_ incluyendo los códigos utilizados, los resultados, los gráficos realizados y comentarios. Debe seguir una estructura similar a un informe (se debe introducir los problemas a trabajar, presentar los resultados y discutirlos). Si lo prefiere puede entregar un _Jupyter Notebook_ por pregunta o uno por toda la tarea, con tal de que todos los entregables esten bien identificados y se encuentren en el mismo repositorio de _Github_.
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el _Jupyter Notebook_ que entregarán. 
* Formato de entrega: envı́o de link del repositorio en _Github_ (en caso de ser repositorio privado, invitar como colaborador al usuario de github "avalderr") al correo electrónico del ayudante (*<alvaro.valderrama.13@sansano.usm.cl>*), en copia al profesor (*<cvalle@inf.utfsm.cl>*). Especificar el siguiente asunto: [INF393/578-2019 Tarea2]
* Fecha de entrega y presentaciones: 31 de Mayo. Hora límite de entrega: 23:00. Cualquier _commit_ luego de la hora límite no será evaluado. Se realizará descuento por atrasos en envío del mail. 

<hr style="height:2px;border:none"/>

La tarea se divide en tres partes:

[1.](#primero) Clasificación de datos no-linealmente separable  
[2.](#segundo) Clasificación con clases desbalanceadas    
[3.](#tercero) Regularización para regresión   

La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo solo son guias y pueden ser creativos al momento de resolver la tarea. Soluciones creativas o elegantes seran valoradas. También en algunas ocaciones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.
Recuerden intercalar su código con comentarios y con celdas _Markdown_ con los comentarios de la pregunta y con cualquier analisis, fórmula (en $ \LaTeX $) o explicación que les parezca relevante para justificar sus procedimientos. 
Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en si, en cambio la argumentación detrás de la elección será lo más ponderado.
Si algun modelo se demora demasiado en correr en su maquina, no olvide que puede correr _Jupyter Notebooks_ en _Collab_ de Google, esto puede ser relevante para las maquinas más lentas al momento de realizar exploraciones con _K-folds_ por ejemplo.

<a id="primero"></a>
## 1. Clasificación de datos no-linealmente separables

Algunos modelos de clasificación buscan implicita o explicitamente separar los datos linealmente, es decir ajustar un hiperplano que separe en 2 subespacios el espacio de las variables, separando ambas clases del _target_. Sin embargo en la mayoría de los casos reales esto resulta imposible. Sin embargo esto no significa que no podamos ajustar modelos con estos datos o buscar transformaciones que nos permitan utilizarlos. 
    
![Alt Text](http://people.cs.uchicago.edu/~dinoj/manifold/swissroll.gif) 

[comment]: <> (Swiss Roll)

En esta primera parte de la tarea crearemos manualmente un set de datos categóricos no linealmente separables. Este se conformará de puntos en $\mathbb{R}^2$, pertenecientes a una de dos categorías etiquetadas 0 o 1, los cuales se encuentran "anidados". Probaremos distintas aproximaciones lineales y no lineales y compararemos sus desempeños.


#### 1.a Creando los datos
La función `do_circles` retorna un conjunto de puntos etiquetados, donde la etiqueta 0 corresponde a puntos siguiendo una circunferencia (con un ruido asociado) y la etiqueta 1 a puntos dentro de esa circunferencia.

Los parametros de esta función tienen nombres intencionalmente no descriptivos. Analise la función y comente interpretación puede darle a cada uno de los parámetros con respecto a la influencia que tendrán en el set de datos obtenidos. Si lo estima conveniente, cambie el nombre de los parámetros a nombres más descriptivos.

In [ ]:
def do_circles(n_dots = 3000, ns = 0.15, scr = 0.3, cf = 1):
    generator = check_random_state(42)
    
    linspace = np.linspace(0, 2*cf*np.pi, n_dots)
    circle_x = np.cos(linspace)
    circle_y = np.sin(linspace)

    inner_circle_x, outer_circle_x, inner_circle_y, outer_circle_y = train_test_split(circle_x, circle_y, test_size = 0.5)
    inner_circle_x , inner_circle_y = inner_circle_x*scr , inner_circle_y*scr 

    X = np.vstack((
        np.append(outer_circle_x, inner_circle_x),
        np.append(outer_circle_y, inner_circle_y)
    )).T

    y = np.hstack([
        np.zeros(n_dots // 2, dtype=np.intp),
        np.ones(n_dots // 2, dtype=np.intp)
    ])

    X += generator.normal(scale = ns, size = X.shape)
    return train_test_split(X, y, test_size=0.3, random_state=42)

#### 1.b Graficar los datos
Cree una función que grafíque los datos, coloreando distintamente los puntos de cada clase. 
Utilice esta función para verificar gráficamente lo que propuso en la pregunta anterior.

In [ ]:
def plot_circles( . . . . ):
    # . . . 
    fig1 = fig.add_subplot(121)
    fig1.scatter(X_train[y_train==0][:,0], X_train[y_train==0][:,1])
    # . . . 
    fig1.set_title("Train Set")
    # . . .

#### 1.c Regresión Logística

Realice una regresión logistica para a lo menos 5000 puntos generados con los siguientes parámetros:
* `do_circles(n_dots, 0.2, 0.3, 1)`
* `do_circles(n_dots, 0, 0.3, 1)`

Grafique los datos que está ajustando y calcule el _score_ del regresor logístico en cada caso.
Qué valor entrega el _score_ del regresor logístico? Le parece un buen desempeño para la tarea a realizar?

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
# . . .

#### 1.d Graficos
Para entender mejor lo que está ocurriendo con el clasificador en cada uno de los casos, grafícaremos los datos clasificados y la frontera de desición ajustada por el modelo. Para esto complete la función siguiente y luego utilicela para ambos modelos y conjuntos de datos ajustados en la pregunta anterior. 

Que puede concluir luego de estas visualizaciones? Es posible mejorar el desempeño de algun clasificador lineal sin hacer una transformación no lineal de los datos? Que ocurre con el clasificador si disminuimos el ruido, mejora realmente su desempeño? 

In [ ]:
def plot_classifier(classifier, X_train, Y_train, X_test, Y_test, train_or_test='test'):
    plt.figure(figsize=(12, 8))
    axis = plt.axes()
    if tr_tst == 'train':
        # Scatter train points (axis.scatter(. . .)) 
    elif tr_tst == 'test':
        # Scatter test points
    XX, YY = np.mgrid[-2:2:200j, -2:2:200j] # grid for cassifier
    Z = clf.predict(np.c_[XX.ravel(), YY.ravel()]) # what the classifier predicts over the space
    Z = Z.reshape(XX.shape) # for consistency
    Zplot = Z > 0.5 # to separate both classes
    axis.pcolormesh(XX, YY, Zplot, cmap='YlGn')
    axis.contour(XX, YY, Zplot, alpha=1, colors=["r"], linestyles=["-"], levels=[0.5])
    plt.show()

Para las siguientes preguntas nos enfocaremos en la **solo en la versión con ruido de los datos** pues en la práctica no existen datos sin ruido. Probaremos algunos métodos no lineales para ver que tanto logramos mejorar el desempeño en nuestros datos. 

#### 1.e SVM

Entrene un clasificador SVM con los datos con ruido. Qué forma tiene la frontera de desición esta vez? 

Mejoran los resultados al alejarse de los metodos puramente lineales? Comente sobre el desempeño del modelo y su sensibilidad a los parámetros `C` y `gamma`. 

In [ ]:
from sklearn.svm import SVC
SV = SVC(C=1, gamma=5)

#### 1.f KNN
Pruebe ahora con otro método no linal, _K nearest neighbors_ o KNN. Discuta sus resultados y la frontera de desición que encuentra el modelo. 

Realice un _Cross Validation_ sobre el parámetro `n_neighbors` o `p` (o ambos usando un mallado) y comente si mejoran significativamente los resultados. Puede apoyarse de los códigos de la tarea 1 para realizar el _Cross Validation_.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=5, p=2)

#### 1.g
Del mejor modelo obtenido en el punto anterior, grafíque los datos de test coloreando de manera difente los puntos mal clasificados por su modelo. Por qué el modelo no logra clasificar bien estos puntos? Qué habría ocurrido si estuvieramos utilizando los datos sin ruido?

In [ ]:
# do it yourself c:

#### 1.h Metodo Kernel
Utilizaremos Kernel Gaussiano para preprocesar los datos, pasando implícitamente por un espacio altamente dimensional (más especificamente por un _manifold_ de un espacio infinito dimensional), obteniendo una representación lo más linealmente separable posible. 

Ajuste el kernel y transforme los datos. Grafique los puntos transformados y comente que realiza el modelo. Varie luego el valor de `gamma` para tratar de entender qué influencia tiene sobre la separabilidad de los puntos transformados. Le parece un buen método para este problema? 

Usando algún valor de `gamma` que le parezca adecuado, ajuste un modelo de regresión logística sobre los datos transformados. Mejora el desempeño del modelo respecto a la versión original? Se logra aumentar la separabilidad lineal de los datos efectivamente? Apoyese de gráficos y métricas adecuados. 

In [ ]:
from sklearn.decomposition import KernelPCA
ker = KernelPCA(2,kernel='rbf',gamma=5)
ker.fit(X_train)
X_ker_train = ker.transform(X_train)
# . . 

#### 1.i Manualmente
Como nosotros conocemos como se generaron los datos tenemos una ventaja sobre los modelos. Genere una transformación de los datos que estime conveniente para obtener datos linealmente separables. Puede aumentar o disminuir la dimensión a gusto, por ejemplo puede realizar algo del estilo $(x,y)\longrightarrow (x+y)$, ó $(x,y)\longrightarrow (x,y,x+y,x-y)$. Apoyese de las carácteristicas geométricas del conjunto de datos y justifique sus elecciónes con gráficos adecuados. 

Luego entrene un regresor logístico sobre sus datos transformados y comente sobre la utilidad de conocer cómo se generaron los datos. 

<a id="segundo"></a>
## 2. Clasificación con clases desbalanceadas

En las tareas de clasificación supervisada, buscamos que mediante los ejemplos etiquetados la maquina pueda aprender los atributos inerentes a las distintas clases. Sin embargo, en muchos casos reales de clasificación, la cantidad de ejemplos de cada clase son muy dispares, en cuyo caso hablamos de clases desbalanceadas. Si uno no aplica estrategias para compensar este problema, la maquina aprenderá muy bien las caracteristicas de la clase más representada pero no logrará extraer información generalizable de la clase menos representada. Además, debemos tener cuidado al momento de interpretar las distintas métricas a las cuales tenemos acceso, pues debemos tomar en cuenta el balance de las clases y la naturaleza del problema para evaluar realmente que tan bueno es el desempeño. 

<img src="http://tynmedia.com/tynmag/wp-content/uploads/sites/3/2018/10/fraude.jpg"  style="height:3cm;"  />

Para esta parte de la tarea utilizaremos el siguiente _dataset_ publicado en Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud. Este conjunto de datos contiene unos 285000 ejemplos de transacciones con tarjetas de credito reales, realizadas en 2013 por clientes europeos. Los datos son totalemente anónimos y son el resultado de un PCA a partir de los datos originales excepto por la columna `Time` con el tiempo en segundos desde la primera transacción, `Amount` con el monto de la transacción y `Class` que indica si la transacción es fraudulenta o no. La tarea en cuestión consiste en lograr predecir cuando una transacción es fraudulenta o no automaticamente, para así poder detenerla antes de que se termine. Sin embargo, por la naturaleza de las transacciones bancarias, este probleme es desbalanceado, de hecho de los 285000 transacciones registradas, solo 492 son fraudulentas! A lo largo de esta pregunta pondremos en evidencia los problemas que se originan de este desbalance de clases y trataremos de dar luces a algunas herramientas que nos permitan sortear esos problemas y como medir realmente el desempeño en esta clase de problemas. 

#### 2.a Carga de datos
Descargue los datos y carguelos usando pandas. Haga una exploración rápida de los datos, cuantos datos hay, como se reptarten sus valores.

Grafique la matriz de correlación, a primera vista parece relevante mantener la columna `Time`? En su opinión, conociendo la naturaleza del problema le parece relevante esa información para predecir el _target_?

Cuantos ejemplos hay de cada clase? Son las clases desbalanceadas efectivamente? 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('creditcard.csv')

df.describe()

# . . . 

plt.imshow(df.corr())

# . . .

df.loc[lambda x: x.Class != 0].shape

#### 2.b Ligero preprocesamiento

Escale la columna `Amount`. Por qué es aconsejable realizar esto? Le parece necesario realizarlo en el resto de las columnas? 

Separe los atributos del _target_.

Separe luego los datos en _Training set_ y _Validation set_, con un 20% de los datos como validación y el resto como _train_. Asegurese que se mantienen las proporciones de ejemplos de cada clase en ambos sets. 

Qué ocurriria si por ejemplo todos los ejemplos de la clase 1 quedaran en el _validation set_, que haría cualquier máquina de aprendizaje al aprender solo con la clase 0?

In [ ]:
from sklearn.preprocessing import StandardScaler

# . . .

df['Amount'] = scaler.transform(df['Amount'].values.reshape(-1,1))

x = df[df.columns[1:-2]]
y = df.Class

from sklearn.model_selection import train_test_split

#### 2.c Regresor logístico

Entrene un regresor logístico con los datos obtenidos de la pregunta anterior y calcule su desempeño (_score_ por ahora) sobre los datos de validación. A priori le parece un buen desempeño? 

In [ ]:
from sklearn.linear_model import LogisticRegression
# . . . 
# . . . 

#### 2.d _Always_ 0

Suponga un modelo le entrega como resultado en el conjunto de validación el siguiente `y_pred`. Qué está haciendo este predictor? Que _score_ obtendría tal modelo sobre el conjunto de validación? Le parece que el modelo tenga un buen desempeño? Concluya sobre la calidad del _accuracy_ que obtenemos con `.score` para evaluar el desempeño de estos modelos. 

In [ ]:
y_pred = pd.np.zeros(y_val.shape)

# . . .

#### 2.e Matriz de confución

Investigue un poco sobre los valores que conforman la matriz de confución o _confusion matrix_. Comente sobre la significancia de los distintos valores para el problema en cuestión. 

Escriba una función que a partir de un modelo o de los valores predecidos por un modelo, grafique la matriz de confución. 

In [ ]:
# Puede usar libreria seaborn para realizar facilmente heatmaps anotados: 
import seaborn as sns
# . . .
sns.heatmap( confusion_matrix(y_test, model.predict(X_test))/len(X_test),cmap='winter', annot=True)    
# . . .

Grafique la matriz de confución para la regresión logística realizada en el punto c y para los valores `y_pred` del punto d. Le parecen parecidos ahora ambos modelos? 

#### 2.f Curva ROC
La curva ROC tiene bastante utilidad para evaluar la calidad de distintos modelos en estos casos. Investigue un poco sobre su significado para poder interpretarla correctamente y luego escriba una función que reciba un modelo entrenado y la grafíque. 

Note que `sklearn` trae implementado una función que entrega los puntos de la curva ROC, note eso sí que debe entregarle las probabilidades usando el método `predict_proba` del modelo entrenado. 

Para evaluar distintos modelos utilizará bastante las curvas ROC y la matriz de confución, por lo que sería recomendable escribir una función que reciba un modelo y datos y realice tanto la curva ROC como la matriz de confución, aunque escribir tal funcion no es requisito. 

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_ts, model.predict_proba(x_ts)[:,1],pos_label=1)

plt.plot(fpr, tpr, label=# . . . )

Utilice la función definida para su modelo de regresión logística. Comente el gráfico obtenido. 

#### 2.g _Undersampling_

Considerando que la gran mayoría del aprendizaje se realiza en los ejemplos no fraudulentos, una aproximación para tratar de mejorar el desempeño de los modelos entrenados es simplemente reducir la cantidad de ejemplos de una clase para obtener nuevos conjuntos de datos con clases más balanceadas. Así, el aprendizaje se realiza de manera más balanceada entre ambas clases y se logran extraer mejor los atributos de cada clase. 

Complete la siguiente función para que realice un _undersampling_ de la clase 0 para obtener una proporción en el nuevo conjundo de datos de `times` veces el numero de transacciónes normales frente a fraudulentas. Comente sobre la decición del parametro `replace` de la función `numpy.random.replace`. 


Note que la función recibe el _DataFrame_ con todos los datos. Se pide que retorne los con juntos de _train_ y _validation_ para luego poder entrenar un modelo. 

In [ ]:
def undersample(df, times = 10, validation_ratio = 0.2):
    fraud_indices = pd.np.array(df[df.Class==1].index)
    normal_indices = # . . .

    Count_Fraud_transacation = # . . .
    
    Normal_indices_undersample = pd.np.array(
    pd.np.random.choice(
        # wich indices . . . ,
        Count_Fraud_transacation * #something,
        replace = # True or False?)
    )
    
    undersample_data = pd.np.concatenate([fraud_indices,Normal_indices_undersample])
    undersample_data = df.iloc[undersample_data]
    
    # . . . X, y
    # train test split
    return x_train, y_train, x_test, y_test

#### 2.h Entrenar con _Undersampling_

Pruebe distintos valores para el parámetro `times` y evalue si hay diferencias significativas en la curva ROC y la matriz de confución. Evidentemente el número de ejemplos cambia a medida cambia el parámetro `times`, por lo que la matriz de confusión debe dividirla por la cantidad de datos considerados para realizar comparaciones justas, de proporciones de ejemplos clasificados y no de número de ejemplos. 

Como se comparan estos modelos con el modelo de regresión logística con todos los datos? Ve alguna ventaja en el uso de _undersampling_ frente al uso de todos los datos? 

#### 2.i SVM
Las SVM son una herramienta muy poderosa pero comparativamente costosa (frente a la regresión lineal por ejemplo). Entrenar una SVM con todos los datos lleva un tiempo considerablemente mayor que lo que se han demorado nuestros modelos hasta ahora, todo ese tiempo si si quiera saber si la SVM se adapta bien a este problema o no! Una ventaja de _undersample_ es precisamente que tenemos menos datos, con lo que podemos probar rápidamente muchos modelos antes de enfocarnos en detalle en uno. 

Entrene una SVM usando _undersampling_ y compare con el desempeño de este modelo frente a un regresor logístico en las mismas condiciones. Vale la pena considerar SVM? 

In [ ]:
from sklearn.svm import SVC
# . . .

#### 2.j _Oversampling_

Otra aproximación para resolver el problema de las clases desbalanceadas es hacer _oversampling_. A partir de una muestra relativamente pequeña de ejemplos, en este caso de los casos de fraude, se busca crear un conjunto de datos más grande similar al inicial, que permita al modelo aprender las caracteristicas de esa clase, buscando nuevamente obtener clases más balanceadas que originalmente, pero esta vez sin reducir la cantidad de ejemplos. Esta aproximación tiene la ventaja que no reduce la cantidad de ejemplos pero trae la complicación de decidir cómo se van a crear los datos nuevos.

Dos aproximaciones bastante estandar son SMOTE y ADASYN. Investigue un poco que hace SMOTE y que diferencia implemente ADASYN. Por qué cree puede ser deseable buscar que los datos sintéticos no sean linealmente dependientes de los datos originales? 

Elija alguno de los dos métodos e impelementelo con el modelo de regresión logística. Comente sobre el desempeño de tal modelo. 

In [ ]:
!pip install  imbalanced-learn

from imblearn.over_sampling import SMOTE, ADASYN
ada = ADASYN()
x_tr_r, y_tr_r  = smo.fit_resample(x_tr,y_tr)


Note que tanto SMOTE como ADASYN tienen muchos parámetros que podrían afectar la calidad de los datos generados. No se requiere que elijan tales parametros (los por defecto bastan), pero se valorará un pequeño comentario sobre los parámetros de los métodos y qué representan en la heuristica utilizada. 

<a id="tercero"></a>
## 3. Regularización para regresión. 

Un problema que podemos encontrar transversalmente a todos los modelos es el problema del _overfitting_. Este problema consiste en un modelo que aprende demasiado bien los datos de ejemplo o entrenamiento, al punto que al momento de ver ejemplos desconocidos o de validación, baja su desempeño respecto a un modelo "menos entrenado". En algunos casos podriamos decir que el modelo "aprende de memoria" los ejemplos, en vez de intentar obtener información generalizable a partir de su entrenamiento. Para el caso de la regresión, una de las aproximaciones más comunes para tratar de mitigar este efecto es el de la regularización. Agregando una penalización a la norma de los parametros de la regresión podemos restringir la eficiencia del modelo al momento de aprender los datos de entrenamiento, incluso forzando dependiendo de la intensidad de la regularización que algunos parametros se vuelvan cero. 

<img src="https://pngimage.net/wp-content/uploads/2018/06/house-for-sale-sign-png-6.png"  style="height:13.9cm;"  />

En esta pregunta utilizaremos nuevamente la base de datos de precios de casas en King County, https://www.kaggle.com/harlfoxem/housesalesprediction, que utilizamos en la pregunta 1. Utilizaremos estos datos ya conocidos y analizados anteriormente para enfocarnos en la regularización para regresión. Exploraremos como varian los distintos errores de predicción y como cambian los valores de los parámetros ajustados por el modelo para distintos valores del coeficiente de regularización, para tratar de entender que ocurre con las distintas regularizaciones y sus diferencias. 

#### 3.a Carga y preprocesamiento.
Cargue y preprocese los datos de la misma forma en que lo realizó en la tarea 1. Puede reciclar sus códigos y agregar modificaciones que le parezcan pertinentes. Para esta parte utilize 0.5 de los datos como _training set_, 0.3 como _validation set_ y 0.2 como _test set_.

#### 3.b Modelo sin regularización
Entrene un modelo de regresión lineal no regularizado con los datos preprocesados. Calcule el error cuadrático medio (MSE) del modelo para el conjunto de validación y el de entrenamiento. 

In [ ]:
regressor = LinearRegression()
regressor.fit(# . . . )
    
mse = ((reg.predict(x_val)-y_val)**2).mean()

#### 3.c Regularización
Discuta brevemente que busca realizar (en terminos del ajuste de los parámetros del modelo de regresión lineal) la regularización de _Lasso_. Por qué se habla de penalización en norma $L^1$? Apoyese de ecuaciones. Qué importancia tiene el parámetro de regularización $\lambda$? 

Qué diferencia hay entre _Lasso_ y _Ridge_? 

#### 3.d Influencia sobre el error
Grafique como varia el error de entrenamiento y el error de validación a medida se cambia el valor del parámetro $\lambda$. Para esto debe entrenar un modelo para cada uno de los parámetros que considerará en el gráfico y calcular el MSE para ambos sets. Comente sobre lo que ve en el gráfico y si lo encuentra pertinente puede calcular en más detalle alguna zona que le parezca interesante. 

In [ ]:
# for example:
MSE_list = []
space = pd.np.logspace(-2,1,50)
for lambd in space:
    model = Lasso(alpha=lambd)
    # fit, error...
# . . . 
# plt.plot( space, MSE_list, . . . ) 

#### 3.e Influencia sobre los coeficientes
Grafique ahora como se comporta el valor de los coeficientes del modelo lineal en el modelo entrenado a medida crece $\lambda$, tanto para _Lasso_ como para _Ridge_. Para esto entrene un modelo para cada valor de $\lambda$ y guarde sus coeficientes, como propone el princio de código a continuación. 

Qué diferencias ve entre _Ridge_ y _Lasso_? Coincide este comportamiento con lo que esperaría teóricamente de cada uno de los modelos? 

Podemos utilizar estos gráficos para discutir sobre la cantidad de información que entrega cada uno de los atributos para predecir el _target_? Cual de ambos (_Ridge_ o _Lasso_) le parece mejor para este tipo de discución? Compare lo que muestran estos gráficos con alguna otra aproximación para medir la significancia de cada atributo (como correlación o _mutual information_ por ejemplo) y discuta. 

In [ ]:
space = pd.np.linspace(0.0001,3,100)
coefs = []
for l in space:
    model = # . . .
    
    # . . .
    
    coefs.append(model.coef_)

for y_arr, label in zip(pd.np.squeeze(coefs).T, x_cols):
    plt.plot(space, y_arr, label=label)
    # . . .

#### 3.f K-_fold_ sobre $\lambda$
Realice 5-_fold_ o 10-_fold_ sobre el parámetro de regularización $\lambda$, ya sea para _Lasso_ o _Ridge_. Explore a lo menos 20 valores distintos de $\lambda$ en un intervalo relevante luego de lo aprendido en las preguntas anteriores. Utilice MSE como el error a minimizar. 

Compare el error de entrenamiento y de validación para este modelo respecto al modelo lineal sin regularización entrenado inicialmente. Comente.